In [28]:
import numpy as np
import os.path as osp

from geo_tool import Point_Cloud

from tf_lab.nips.helper import pclouds_centered_and_half_sphere
from tf_lab.iclr.helper import load_multiple_version_of_pcs
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

from pcloud_benchmark.evaluate_gan import entropy_of_occupancy_grid, jensen_shannon_divergence

In [29]:
%load_ext autoreload
%matplotlib inline
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
# Parameters
class_name = 'sofa'
n_pc_samples = 2048
cmp_in_sphere = True
voxel_resolution = 28

In [51]:
# Load Wu Data, compute grid variable.
top_in_data = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/mit_wu_isosurface_extracted/'
wu_data = np.load(osp.join(top_in_data, class_name + '.npz'))
wu_data = wu_data[wu_data.keys()[0]]
wu_data = pclouds_centered_and_half_sphere(wu_data)
assert(wu_data.shape[1] == n_pc_samples)

_, wu_grid_var = entropy_of_occupancy_grid(wu_data, voxel_resolution, in_sphere=cmp_in_sphere)

In [52]:
# Load GT Data
syn_id = snc_category_to_synth_id()[class_name]
in_data = load_multiple_version_of_pcs('uniform_one', syn_id, n_classes=1)
train_data = in_data['train'].point_clouds
test_data = in_data['test'].point_clouds
val_data = in_data['val'].point_clouds
all_pclouds = np.vstack([train_data, test_data, val_data])

Loading test data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/04256520/85_5_10/test.txt
318 pclouds were loaded. They belong in 1 shape-classes.
Loading train data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/04256520/85_5_10/train.txt
2697 pclouds were loaded. They belong in 1 shape-classes.
Loading val data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/04256520/85_5_10/val.txt
158 pclouds were loaded. They belong in 1 shape-classes.


In [53]:
data_for_cmp = pclouds_centered_and_half_sphere(all_pclouds)
_, gt_grid_var = entropy_of_occupancy_grid(data_for_cmp, voxel_resolution, in_sphere=cmp_in_sphere)

In [54]:
res = jensen_shannon_divergence(wu_grid_var, gt_grid_var)
print res

0.180281468315
